# Moving Average

* Append with test data
* Make lag1-3 features groupby siteid, forecastid

In [3]:
import numpy as np
import pandas as pd
import os 

In [16]:
def data_reader():
    """read data"""
    
    data_dir = os.path.join('..', 'data') 

    train_path = os.path.join(data_dir, 'train.csv')
#     meta_path = os.path.join(data_dir, 'metadata.csv')
#     holiday_path = os.path.join(data_dir, 'holidays.csv')
#     frequency_path = os.path.join(data_dir, 'submission_frequency.csv')    
    test_path = os.path.join(data_dir, 'submission_format.csv')        
#     weather_path = os.path.join(data_dir, 'weather.csv')

    train = pd.read_csv(train_path)
#     meta = pd.read_csv(meta_path)
#     holiday = pd.read_csv(holiday_path)
#     frequency = pd.read_csv(frequency_path)
    test = pd.read_csv(test_path)
#     weather = pd.read_csv(weather_path)
    
    return train, test

In [5]:
train, test = data_reader()

steps:
    1. keep the last lag terms for the train data
    2. combine the filtered train data with test data sorted by siteId, timestamp and forecastId
    3. calculate the moving average for the last lag terms
    4. drop rows from train data

In [15]:
def ma_data(train, test, lag):
    """    
    Parameters
    ----------
    train: DataFrame
    test: DataFrame    
    lag: int
        indicate the number of lag terms to use
        
    Returns
    -------
    ma_result: DataFrame
        a dataframe ready for submission, the value calculated using the last lag terms
    """
    
    train['isTrain'] = True
    test['isTrain'] = False
    
    # keep the last lag terms for the train data
    train_tail = train.groupby('ForecastId').tail(lag)
    
    # combine the filtered train data with test data sorted by siteId, timestamp and forecastId
    ma_df = pd.concat([train_tail[test.columns], test]).sort_values(by=['SiteId','Timestamp','ForecastId'], ascending=True).reset_index(drop=True)

    return ma_df

def calculate_ma(ma_df, lag):
    """
    calcuate the moving average of using the last n terms.
    """
    
    for index, row in ma_df.iterrows():
        if row['isTrain']==False:
            ma_df.loc[index, 'Value'] = ma_df.loc[index-lag:index-1,'Value'].mean()
    
    return ma_df

def print_full(x):
    """
    print the full pandas dataframe
    """
    
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [12]:
lag = 10
ma_df = ma_data(train, test, lag)
ma_df.head(30)

,obs_id,SiteId,Timestamp,ForecastId,Value,isTrain
0,3796431,1,2015-08-19 00:00:00,1,9.973302e+06,True
1,3744244,1,2015-08-20 00:00:00,1,7.450722e+06,True
2,1164577,1,2015-08-21 00:00:00,1,8.673263e+06,True
3,5424559,1,2015-08-22 00:00:00,1,9.223805e+06,True
4,3738309,1,2015-08-23 00:00:00,1,8.615291e+06,True
5,2577489,1,2015-08-24 00:00:00,1,7.640066e+06,True
6,7061479,1,2015-08-25 00:00:00,1,7.805785e+06,True
7,2872820,1,2015-08-26 00:00:00,1,9.160577e+06,True
8,6129661,1,2015-08-27 00:00:00,1,7.307600e+06,True
9,6348748,1,2015-08-28 00:00:00,1,8.092366e+06,True


In [14]:
ma_result = calculate_ma(ma_df, lag)
ma_result.head()

In [ ]:
# ma_result.isnull().sum()/ma_result.shape[0]
# ma_result.to_csv('../submission/ma.csv', sep=',', index=False)